In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Hurricane_Harvey

/content/drive/MyDrive/Hurricane_Harvey


In [ ]:
#deeplabv3+ architecture with the backbone mobilenetv2
#based on: https://github.com/bubbliiiing/deeplabv3-plus-pytorch
import os
import datetime

import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim as optim
from torch.utils.data import DataLoader

from nets.deeplabv3_plus import DeepLab
from nets.deeplabv3_training import (get_lr_scheduler, set_optimizer_lr,
                                     weights_init)
from utils.callbacks import LossHistory, EvalCallback
from utils.dataloader import DeeplabDataset, deeplab_dataset_collate
from utils.utils import download_weights, show_config
from utils.utils_fit import fit_one_epoch


if __name__ == "__main__":
    # Use GPU
    Cuda            = True
    # Whether to use single-machine multi-card distributed operation
    distributed     = False
    # Whether to use sync_bn
    sync_bn         = False
    # Whether to use mixed precision training, can save GPU resource
    fp16            = True
    # As defined in the problem
    num_classes     = 27
    # backbone network
    backbone        = "mobilenet"
    # Doesn't matter here
    pretrained      = False
    # The backbone model dictionary path
    model_path      = "/content/drive/MyDrive/Hurricane_Harvey/model_data/deeplab_mobilenetv2.pth"
    # Downsample_factor
    downsample_factor   = 16
    # Image size
    input_shape         = [512, 512]
    
    # Training epochs
    Init_Epoch          = 0
    Freeze_Epoch        = 50
    Freeze_batch_size   = 8
    
    UnFreeze_Epoch      = 100
    Unfreeze_batch_size = 4
    
    Freeze_Train        = True

    #Learning rate
    Init_lr             = 7e-3
    Min_lr              = Init_lr * 0.01

    #Other hyperparameters
    optimizer_type      = "sgd"
    momentum            = 0.9
    weight_decay        = 1e-4
    
    #Learning rate decay method
    lr_decay_type       = 'cos'

    #Save the model state dictionary for each 5 epochs
    save_period         = 5

    #The path to save the model state dictionary
    save_dir            = 'logs'

    #Whether to evaluate and how often
    eval_flag           = True
    eval_period         = 5

    #------------------------------------------------------------------#
    #The dataset path
    #------------------------------------------------------------------#
    VOCdevkit_path  = '/content/drive/MyDrive/Hurricane_Harvey/VOCdevkit/'

    #Whether to use dice loss
    dice_loss       = False
    #Whether to use focal loss
    focal_loss      = False

    #Whether to weight different classes
    cls_weights     = np.ones([num_classes], np.float32)
    #multi-workers
    num_workers         = 4

    #GPU
    ngpus_per_node  = torch.cuda.device_count()
    if distributed:
        dist.init_process_group(backend="nccl")
        local_rank  = int(os.environ["LOCAL_RANK"])
        rank        = int(os.environ["RANK"])
        device      = torch.device("cuda", local_rank)
        if local_rank == 0:
            print(f"[{os.getpid()}] (rank = {rank}, local_rank = {local_rank}) training...")
            print("Gpu Device Count : ", ngpus_per_node)
    else:
        device          = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        local_rank      = 0

    #Download pretrained weights
    if pretrained:
        if distributed:
            if local_rank == 0:
                download_weights(backbone)  
            dist.barrier()
        else:
            download_weights(backbone)

    model   = DeepLab(num_classes=num_classes, backbone=backbone, downsample_factor=downsample_factor, pretrained=pretrained)
    if not pretrained:
        weights_init(model)
    if model_path != '':
        if local_rank == 0:
            print('Load weights {}.'.format(model_path))
        
        #Load the pretrained dictionary
        model_dict      = model.state_dict()
        pretrained_dict = torch.load(model_path, map_location = device)
        load_key, no_load_key, temp_dict = [], [], {}
        for k, v in pretrained_dict.items():
            if k in model_dict.keys() and np.shape(model_dict[k]) == np.shape(v):
                temp_dict[k] = v
                load_key.append(k)
            else:
                no_load_key.append(k)
        model_dict.update(temp_dict)
        model.load_state_dict(model_dict)
        #print unmatched keys
        if local_rank == 0:
            print("\nSuccessful Load Key:", str(load_key)[:500], "……\nSuccessful Load Key Num:", len(load_key))
            print("\nFail To Load Key:", str(no_load_key)[:500], "……\nFail To Load Key num:", len(no_load_key))
            print("\n\033[1;33;44mHint: It is normal that the head part is not loaded, and it is an error that the backbone part is not loaded.\033[0m")

    #Record loss

    if local_rank == 0:
        time_str        = datetime.datetime.strftime(datetime.datetime.now(),'%Y_%m_%d_%H_%M_%S')
        log_dir         = os.path.join(save_dir, "loss_" + str(time_str))
        loss_history    = LossHistory(log_dir, model, input_shape=input_shape)
    else:
        loss_history    = None

    if fp16:
        from torch.cuda.amp import GradScaler as GradScaler
        scaler = GradScaler()
    else:
        scaler = None

    model_train     = model.train()

    if sync_bn and ngpus_per_node > 1 and distributed:
        model_train = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model_train)
    elif sync_bn:
        print("Sync_bn is not support in one gpu or not distributed.")

    if Cuda:
        if distributed:
            model_train = model_train.cuda(local_rank)
            model_train = torch.nn.parallel.DistributedDataParallel(model_train, device_ids=[local_rank], find_unused_parameters=True)
        else:
            model_train = torch.nn.DataParallel(model)
            cudnn.benchmark = True
            model_train = model_train.cuda()
    
    #Read in the txt file which contain the name of the training and validation images
    with open(os.path.join(VOCdevkit_path, "VOC2007/ImageSets/Segmentation/train.txt"),"r") as f:
        train_lines = f.readlines()
    with open(os.path.join(VOCdevkit_path, "VOC2007/ImageSets/Segmentation/val.txt"),"r") as f:
        val_lines = f.readlines()
    num_train   = len(train_lines)
    num_val     = len(val_lines)

    if local_rank == 0:
        show_config(
            num_classes = num_classes, backbone = backbone, model_path = model_path, input_shape = input_shape, \
            Init_Epoch = Init_Epoch, Freeze_Epoch = Freeze_Epoch, UnFreeze_Epoch = UnFreeze_Epoch, Freeze_batch_size = Freeze_batch_size, Unfreeze_batch_size = Unfreeze_batch_size, Freeze_Train = Freeze_Train, \
            Init_lr = Init_lr, Min_lr = Min_lr, optimizer_type = optimizer_type, momentum = momentum, lr_decay_type = lr_decay_type, \
            save_period = save_period, save_dir = save_dir, num_workers = num_workers, num_train = num_train, num_val = num_val
        )
        wanted_step = 1.5e4 if optimizer_type == "sgd" else 0.5e4
        total_step  = num_train // Unfreeze_batch_size * UnFreeze_Epoch
        if total_step <= wanted_step:
            if num_train // Unfreeze_batch_size == 0:
                raise ValueError('The dataset is too small for training, please expand the dataset.')
            wanted_epoch = wanted_step // (num_train // Unfreeze_batch_size) + 1
            print("\n\033[1;33;44m[Warning] When using the %s optimizer, it is recommended to set the total training step size to above %d. \033[0m"%(optimizer_type, wanted_step))
            print("\033[1;33;44m[Warning] The total amount of training data for this run is %d, the Unfreeze_batch_size is %d, a total of %d Epochs are trained, and the total training step size is calculated as %d. \033[0m"%(num_train, Unfreeze_batch_size, UnFreeze_Epoch, total_step))
            print("\033[1;33;44m[Warning] Since the total training step size is %d, which is less than the recommended total step size %d, it is recommended to set the total generation to %d. \033[0m"%(total_step, wanted_step, wanted_epoch))
        
    if True:
        UnFreeze_flag = False
        #Freeze certain part of training
        if Freeze_Train:
            for param in model.backbone.parameters():
                param.requires_grad = False

        #If you do not freeze training, directly set batch_size to Unfreeze_batch_size.
        batch_size = Freeze_batch_size if Freeze_Train else Unfreeze_batch_size

        #Based on the current batch_size, adaptively adjust the learning rate
        #Define the minimum and maximum learning rate
        nbs             = 16
        lr_limit_max    = 5e-4 if optimizer_type == 'adam' else 1e-1
        lr_limit_min    = 3e-4 if optimizer_type == 'adam' else 5e-4
        if backbone == "xception":
            lr_limit_max    = 1e-4 if optimizer_type == 'adam' else 1e-1
            lr_limit_min    = 1e-4 if optimizer_type == 'adam' else 5e-4
        Init_lr_fit     = min(max(batch_size / nbs * Init_lr, lr_limit_min), lr_limit_max)
        Min_lr_fit      = min(max(batch_size / nbs * Min_lr, lr_limit_min * 1e-2), lr_limit_max * 1e-2)

        #Choosing optimizer type
        optimizer = {
            'adam'  : optim.Adam(model.parameters(), Init_lr_fit, betas = (momentum, 0.999), weight_decay = weight_decay),
            'sgd'   : optim.SGD(model.parameters(), Init_lr_fit, momentum = momentum, nesterov=True, weight_decay = weight_decay)
        }[optimizer_type]

        #Get the formula for the learning rate drop
        lr_scheduler_func = get_lr_scheduler(lr_decay_type, Init_lr_fit, Min_lr_fit, UnFreeze_Epoch)
        
        #Calculate the training steps
        epoch_step      = num_train // batch_size
        epoch_step_val  = num_val // batch_size
        
        if epoch_step == 0 or epoch_step_val == 0:
            raise ValueError("The dataset is too small to continue training, please expand the dataset.")

        train_dataset   = DeeplabDataset(train_lines, input_shape, num_classes, True, VOCdevkit_path)
        val_dataset     = DeeplabDataset(val_lines, input_shape, num_classes, False, VOCdevkit_path)

        if distributed:
            train_sampler   = torch.utils.data.distributed.DistributedSampler(train_dataset, shuffle=True,)
            val_sampler     = torch.utils.data.distributed.DistributedSampler(val_dataset, shuffle=False,)
            batch_size      = batch_size // ngpus_per_node
            shuffle         = False
        else:
            train_sampler   = None
            val_sampler     = None
            shuffle         = True

        gen             = DataLoader(train_dataset, shuffle = shuffle, batch_size = batch_size, num_workers = num_workers, pin_memory=True,
                                    drop_last = True, collate_fn = deeplab_dataset_collate, sampler=train_sampler)
        gen_val         = DataLoader(val_dataset  , shuffle = shuffle, batch_size = batch_size, num_workers = num_workers, pin_memory=True, 
                                    drop_last = True, collate_fn = deeplab_dataset_collate, sampler=val_sampler)

        #Recoding the evaluation results
        if local_rank == 0:
            eval_callback   = EvalCallback(model, input_shape, num_classes, val_lines, VOCdevkit_path, log_dir, Cuda, \
                                            eval_flag=eval_flag, period=eval_period)
        else:
            eval_callback   = None
        
        #Start training
        for epoch in range(Init_Epoch, UnFreeze_Epoch):
            #If the model has a frozen learning part, unfreeze and set parameters
            if epoch >= Freeze_Epoch and not UnFreeze_flag and Freeze_Train:
                batch_size = Unfreeze_batch_size

                ##Based on the current batch_size, adaptively adjust the learning rate
                #Define the minimum and maximum learning rate
                nbs             = 16
                lr_limit_max    = 5e-4 if optimizer_type == 'adam' else 1e-1
                lr_limit_min    = 3e-4 if optimizer_type == 'adam' else 5e-4
                if backbone == "xception":
                    lr_limit_max    = 1e-4 if optimizer_type == 'adam' else 1e-1
                    lr_limit_min    = 1e-4 if optimizer_type == 'adam' else 5e-4
                Init_lr_fit     = min(max(batch_size / nbs * Init_lr, lr_limit_min), lr_limit_max)
                Min_lr_fit      = min(max(batch_size / nbs * Min_lr, lr_limit_min * 1e-2), lr_limit_max * 1e-2)
                ##Based on the current batch_size, adaptively adjust the learning rate
                #Define the minimum and maximum learning rate
                lr_scheduler_func = get_lr_scheduler(lr_decay_type, Init_lr_fit, Min_lr_fit, UnFreeze_Epoch)
                    
                for param in model.backbone.parameters():
                    param.requires_grad = True
                            
                epoch_step      = num_train // batch_size
                epoch_step_val  = num_val // batch_size

                if epoch_step == 0 or epoch_step_val == 0:
                    raise ValueError("The dataset is too small to continue training, please expand the dataset.")

                if distributed:
                    batch_size = batch_size // ngpus_per_node

                gen             = DataLoader(train_dataset, shuffle = shuffle, batch_size = batch_size, num_workers = num_workers, pin_memory=True,
                                            drop_last = True, collate_fn = deeplab_dataset_collate, sampler=train_sampler)
                gen_val         = DataLoader(val_dataset  , shuffle = shuffle, batch_size = batch_size, num_workers = num_workers, pin_memory=True, 
                                            drop_last = True, collate_fn = deeplab_dataset_collate, sampler=val_sampler)

                UnFreeze_flag = True

            if distributed:
                train_sampler.set_epoch(epoch)

            set_optimizer_lr(optimizer, lr_scheduler_func, epoch)

            fit_one_epoch(model_train, model, loss_history, eval_callback, optimizer, epoch, 
                    epoch_step, epoch_step_val, gen, gen_val, UnFreeze_Epoch, Cuda, dice_loss, focal_loss, cls_weights, num_classes, fp16, scaler, save_period, save_dir, local_rank)

            if distributed:
                dist.barrier()

        if local_rank == 0:
            loss_history.writer.close()

initialize network with normal type
Load weights /content/drive/MyDrive/Hurricane_Harvey/model_data/deeplab_mobilenetv2.pth.

Successful Load Key: ['backbone.features.0.0.weight', 'backbone.features.0.1.weight', 'backbone.features.0.1.bias', 'backbone.features.0.1.running_mean', 'backbone.features.0.1.running_var', 'backbone.features.0.1.num_batches_tracked', 'backbone.features.1.conv.0.weight', 'backbone.features.1.conv.1.weight', 'backbone.features.1.conv.1.bias', 'backbone.features.1.conv.1.running_mean', 'backbone.features.1.conv.1.running_var', 'backbone.features.1.conv.1.num_batches_tracked', 'backbone.features.1.conv.3.weight', 'bac ……
Successful Load Key Num: 369

Fail To Load Key: ['cls_conv.weight', 'cls_conv.bias'] ……
Fail To Load Key num: 2

温馨提示，head部分没有载入是正常现象，Backbone部分没有载入是错误的。
Configurations:
----------------------------------------------------------------------
|                     keys |                                   values|
-------------------------------------

Epoch 1/100: 100%|██████████| 302/302 [03:43<00:00,  1.35it/s, f_score=0.36, lr=0.00035, total_loss=2.06]


Finish Train
Start Validation


Epoch 1/100: 100%|██████████| 33/33 [00:25<00:00,  1.32it/s, f_score=0.316, lr=0.00035, val_loss=2.17]


Finish Validation
Epoch:1/100
Total Loss: 2.057 || Val Loss: 2.167 
Save best model to best_epoch_weights.pth
Start Train


...

Finish Train
Start Validation


Epoch 100/100: 100%|██████████| 67/67 [00:05<00:00, 12.01it/s, f_score=0.619, lr=1.75e-5, val_loss=0.545]


Finish Validation
Get miou.


100%|██████████| 270/270 [00:13<00:00, 20.18it/s]


Calculate miou.
Num classes 27
===> mIoU: 28.96; mPA: 36.72; Accuracy: 81.37
Get miou done.
Epoch:100/100
Total Loss: 0.373 || Val Loss: 0.545 
